In [85]:
import pandas as pd 
import numpy as np
import psycopg2 as psy2
import psycopg2.extras as extras

from contextlib import closing

In [ ]:
# pip install fuzzywuzzy python-Levenshtein

In [75]:
emails = pd.read_excel("./Libro2.xlsx")
emails = emails.iloc[:,1:]
emails['formato'] = 'ALL'
emails

,RECINTO,CARGO,NOMBRE,CORREO,formato
0,ALTO LAS CONDES,EM,Francisco Alfaro,falfarot@ripley.com,ALL
1,ALTO LAS CONDES,JPP,Nicholas Rodriguez,nrodriguezc@ripley.com,ALL
2,ALTO LAS CONDES,GT,Daniela Donoso,ddonosop@ripley.com,ALL
3,ANTOFAGASTA,EM,Solemir Sandoval,g-mantantofagasta@ripley.com,ALL
4,ANTOFAGASTA,JPP,Rolando Vera,rveram@ripley.com,ALL
...,...,...,...,...,...
134,VALPARAISO,JPP,Agustin Valencia,avalencia@ripley.com,ALL
135,VALPARAISO,GT,Francisco Balboltin,fbalbontinv@ripley.com,ALL
136,VIÑA SUCRE,EM,Marco Coral,mcorralz@ripley.com,ALL
137,VIÑA SUCRE,JPP,Franco Araya,farayaz@ripley.com,ALL


In [76]:
emails['CORREO'] = emails['CORREO'].apply(lambda x : str(x).strip())

In [77]:
print(f"TOTAL CORREOS : {emails.shape[0]}")

TOTAL CORREOS : 139


In [78]:
#conexion 
credenciales = {
    'host' : 'efizity-gestion-energetica.cluster-cb38cbcdstoa.us-east-1.rds.amazonaws.com',
    'user' : 'gestion',
    'password' : 'KJXH9tcPpr1dMqE3jsWn',
    'database' : 'alertas'
}

In [86]:

def fuzzy_match(value, choices):
    
    import pandas as pd
    from fuzzywuzzy import process

    result = process.extractOne(value, choices)
    
    if result:
        best_match, score, _ = result 
        if score >= 80:
            return best_match
    return None  

def validate_conexion(cliente : str)->int:
    """
    Objetivo : Validar que existe un conexion entre el script y la base de datos. 
    """    
    
    test_query = f"""
    SELECT * FROM emails_sobreconsumo.{cliente}
    """
    
    with closing(psy2.connect(**credenciales)) as cnx:
        with closing(cnx.cursor()) as cursor:
            cursor.execute(test_query)
            results = cursor.fetchall()
            columns = [x[0] for x in cursor.description]
            data_db = pd.DataFrame(results,columns=columns)
            if len(results) > 0: 
                print(f"funciona la conexion")
                return 1,data_db
            else: 
                return 0

def validate_email(cliente : str,email : str)->int:

    #continuar con el codigo
    query = f"""SELECT email FROM emails_sobreconsumo.{cliente} where email = '{email}'"""
    with closing(psy2.connect(**credenciales)) as cnx:
        with closing(cnx.cursor()) as cursor:
            cursor.execute(query)
            result = cursor.fetchall()
            # print(result)
            if len(result) > 0:
                return 1 # existe
            else:
                return 0 # no existe

def update_values():
    pass


def insert_new_values(data : pd.DataFrame,cliente : str):
    estado_cnx,data_db = validate_conexion(cliente)
    print(f"total correos en db {data_db.shape[0]}")
    
    #correos no existentes
    correo_ne = data.loc[~data['CORREO'].isin(data_db['email'].unique()),]
    print(f"total correos no existentes : {correo_ne.shape[0]}")
    correo_ne['cenco'] = correo_ne['RECINTO'].apply(lambda x: fuzzy_match(x, data_db.loc[data_db['cenco']!='ALL','cenco']))

    df_final = correo_ne[['cenco','formato','NOMBRE','CORREO']]
    df_final = df_final.astype('str')
    data = [tuple(x) for x in df_final.to_records(index=False)]

    insert_query = f"INSERT INTO emails_sobreconsumo.{cliente} VALUES %s"
    
    with closing(psy2.connect(**credenciales)) as connection:
            with closing(connection.cursor()) as cursor:
                try:
                    extras.execute_values(cursor, insert_query, data)
                    connection.commit()
                except (Exception, psy2.DatabaseError, ValueError) as err:
                    print("ERROR! ", err)
                    connection.rollback()
                finally:
                        print("se cargaron los datos")    

    # if estado_cnx == 1:
    #     #correr por cada correo y validar su existencia
    #     for c in corre_ne['CORREO'].unique():
    #         #insertar nuevo valor


In [87]:
#ejecutar
#validar conexion 
insert_new_values(emails,'ripley')

funciona la conexion
total correos en db 146
total correos no existentes : 8


/var/folders/sh/pm7v0y8954z4h0ks1rc58ry80000gn/T/ipykernel_7434/4010608719.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correo_ne['cenco'] = correo_ne['RECINTO'].apply(lambda x: fuzzy_match(x, data_db.loc[data_db['cenco']!='ALL','cenco']))


se cargaron los datos
